# 🔴 Lab 1 — Evasion Attack
### Certified AI Penetration Tester – Red Team (CAIPT-RT)

---

## 🎯 The Story

Imagine you are a spammer. You want to send promotional messages to thousands of people, but there is a spam filter standing in your way — a machine learning model trained to detect spam and block it before it reaches anyone's inbox.

Your goal as an attacker is not to break the filter or hack into it. Instead, you want to **craft your spam message so cleverly that the filter does not recognise it as spam**, even though any human reading it clearly would.

This is called an **Evasion Attack**. You are evading detection by manipulating your input.

---

## 📖 What is an Evasion Attack?

An evasion attack happens **after** a model has already been trained and deployed. The attacker does not touch the model or its training data. Instead, the attacker carefully modifies an input just enough to fool the model while keeping it meaningful to a human.

**Real world examples:**
- Tweaking spam emails to bypass spam filters
- Modifying malware signatures slightly to bypass AI-based antivirus software
- Altering network traffic patterns to evade AI intrusion detection systems

---

## 🗂️ What We Will Do in This Lab

1. Load a real SMS spam dataset and explore it
2. Convert text messages into numbers the model can understand
3. Train a spam classifier — this is the model we will attack
4. Confirm the classifier works before attacking it
5. Use ART to perform an evasion attack
6. Measure and reflect on the results

---

## ⚙️ Step 1: Import the Tools We Need

In [ ]:
# =============================================================================
# IMPORTS
# =============================================================================
# numpy      : handles numbers and arrays
# pandas     : loads and explores datasets
# sklearn    : builds and trains machine learning models
# art        : the Adversarial Robustness Toolbox - our attack toolkit
# matplotlib : draws charts and graphs
#
# We suppress the PyTorch warning from ART since we are not using PyTorch.
# =============================================================================

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from art.estimators.classification import SklearnClassifier
from art.attacks.evasion import HopSkipJump

np.random.seed(42)
print("All tools imported successfully. Ready to begin.")

---

## 📂 Step 2: Load and Explore the Dataset

We are using the **SMS Spam Collection dataset** — 5,574 real SMS messages each labeled as either `spam` or `ham` (ham means legitimate).

Before training anything, we always explore the data first. A good attacker — and a good defender — understands the data they are working with.

In [ ]:
# =============================================================================
# LOAD THE DATASET
# =============================================================================
# The SMSSpamCollection file is tab-separated — each line has two pieces
# of information separated by a tab: the label (spam or ham) and the message.
# =============================================================================

df = pd.read_csv(
    '../datasets/SMSSpamCollection',
    sep='\t',
    header=None,
    names=['label', 'message'],
    encoding='latin-1'
)

print(f"Dataset loaded successfully.")
print(f"Total messages: {len(df)}")
print("")
print("First 5 rows:")
print("-" * 60)
df.head()

### 👀 What Do You See?

Look at the first 5 rows printed above.

- What are the two columns in this dataset?
- Can you tell just by reading the messages which ones are spam?
- What patterns do you notice in the spam messages?

---

In [ ]:
# =============================================================================
# EXPLORE THE DATASET
# =============================================================================
# How many spam vs legitimate messages are there?
# This matters because a model that guesses "legitimate" every single time
# would score well on accuracy but be completely useless as a spam filter.
# =============================================================================

label_counts = df['label'].value_counts()

print("Message counts by label:")
print("-" * 30)
print(f"Legitimate (ham) : {label_counts['ham']}")
print(f"Spam             : {label_counts['spam']}")
print(f"Total            : {len(df)}")
print("")
print(f"Spam percentage  : {label_counts['spam']/len(df)*100:.1f}%")
print(f"Ham percentage   : {label_counts['ham']/len(df)*100:.1f}%")

plt.figure(figsize=(6, 4))
label_counts.plot(kind='bar', color=['steelblue', 'tomato'])
plt.title('Distribution of Spam vs Legitimate Messages')
plt.xlabel('Label')
plt.ylabel('Number of Messages')
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig('../outputs/lab1_dataset_distribution.png')
plt.show()
print("Chart saved to outputs folder.")

### 👀 What Do You See?

- What percentage of messages are spam?
- If a filter labeled every message as legitimate without reading it, what accuracy would it get? Is that a good spam filter?
- Why does the balance between spam and legitimate messages matter when training a model?

---

## 🔢 Step 3: Convert Text to Numbers

Machine learning models cannot read text — they only understand numbers. We use **TF-IDF (Term Frequency - Inverse Document Frequency)** to convert each message into a list of numbers.

The idea is simple: words that appear a lot in spam messages (like FREE, WIN, PRIZE) get high scores. Words that appear in almost every message (like the, a, is) get low scores because they are not useful for identifying spam.

In [ ]:
# =============================================================================
# CONVERT TEXT TO NUMBERS USING TF-IDF
# =============================================================================
# max_features=5000 means we keep the 5000 most useful words.
# Each message becomes a vector of 5000 numbers.
# =============================================================================

df['label_num'] = df['label'].map({'spam': 1, 'ham': 0})

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['message']).toarray()
y = df['label_num'].values

print(f"Each message is now a vector of {X.shape[1]} numbers.")
print(f"Total messages converted: {X.shape[0]}")
print("")
print(f"Example - first message:")
print(f"  Original : {df['message'][0]}")
print(f"  Label    : {df['label'][0]}")
print(f"  Non-zero values in vector: {np.count_nonzero(X[0])} out of {X.shape[1]}")

### 👀 What Do You See?

- Each message has been converted into 5000 numbers. Why do most of those numbers equal zero?
- Why can the model not just read the words directly?

---

## ✂️ Step 4: Split Data into Training and Testing Sets

We split data into two groups:

**Training set** — the examples the model learns from. Like study material.

**Testing set** — examples the model has never seen, used to check if it actually learned. Like an exam.

If we tested on the same data used for training, the model would just memorise answers — like a student who gets the exact same questions on the exam as in their homework.

In [ ]:
# =============================================================================
# SPLIT INTO TRAINING AND TESTING SETS
# =============================================================================
# test_size=0.2  : 80% training, 20% testing
# stratify=y     : keeps same spam/ham ratio in both sets
# random_state=42: everyone gets the same split (reproducible)
# =============================================================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Data split complete.")
print("-" * 40)
print(f"Training set : {len(X_train)} messages")
print(f"Testing set  : {len(X_test)} messages")
print("")
print(f"Training spam : {sum(y_train == 1)} | Training ham : {sum(y_train == 0)}")
print(f"Testing spam  : {sum(y_test == 1)}  | Testing ham  : {sum(y_test == 0)}")

### 👀 What Do You See?

- Is the spam-to-ham ratio similar in both sets? Why does that matter?

---

## 🏋️ Step 5: Train the Spam Filter Model

Now we train the spam filter. We use **Logistic Regression** — a reliable model for text classification.

Training means the model studies thousands of labeled messages and learns patterns. It figures out that messages with FREE, WINNER, URGENT tend to be spam, while messages with home, later, dinner tend to be legitimate.

In [ ]:
# =============================================================================
# TRAIN THE SPAM FILTER
# =============================================================================
# LogisticRegression learns a mathematical boundary between spam and ham.
# fit() is where training actually happens.
# We pass it X_train (message vectors) and y_train (correct labels).
# =============================================================================

print("Training the spam filter...")
print("")

model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Training complete!")
print("")
print(f"Accuracy on test set: {accuracy * 100:.2f}%")
print("")
print("Detailed breakdown:")
print("-" * 50)
print(classification_report(y_test, y_pred, target_names=['Ham (legitimate)', 'Spam']))

### 👀 What Do You See?

- What accuracy did the model achieve? Is that high enough to trust?
- Look at precision and recall for spam. Precision = of all messages it called spam, how many actually were? Recall = of all actual spam, how many did it catch?
- Which is more dangerous — low precision (too many false alarms) or low recall (missing real spam)?

---

## 🎯 Step 6: Test the Model Manually

Before attacking, confirm the model works by testing it on messages you write yourself.

In [ ]:
# =============================================================================
# MANUALLY TEST THE SPAM FILTER
# =============================================================================

def classify_message(message):
    vec = vectorizer.transform([message]).toarray()
    prediction = model.predict(vec)[0]
    confidence = model.predict_proba(vec)[0][1]
    label = "SPAM" if prediction == 1 else "HAM (legitimate)"
    return label, confidence

test_messages = [
    "Congratulations! You have WON a FREE iPhone! Click here to claim your PRIZE now!",
    "Hey, are you coming to dinner tonight? Let me know!",
    "URGENT: Your account has been suspended. Call us immediately to verify your details.",
    "Can you pick up some milk on your way home?",
    "FREE entry into our weekly competition to win FA Cup Final tickets!"
]

print("Manual spam filter test:")
print("=" * 70)
for msg in test_messages:
    label, confidence = classify_message(msg)
    display = msg[:60] + "..." if len(msg) > 60 else msg
    print(f"Message  : {display}")
    print(f"Decision : {label} (spam confidence: {confidence:.2%})")
    print("-" * 70)

### 👀 What Do You See?

- Did the model correctly identify the spam and legitimate messages?
- Which messages was the model most confident about?

### 🧪 Try This

Edit the cell below and write your own test message. Try:
1. An obvious spam message
2. A borderline message that could go either way
3. A message that looks like spam but is actually legitimate

In [ ]:
# =============================================================================
# TRY IT YOURSELF - change the message below and run this cell
# =============================================================================

my_message = "Type your own message here and see what happens!"

label, confidence = classify_message(my_message)
print(f"Your message : {my_message}")
print(f"Decision     : {label}")
print(f"Confidence   : {confidence:.2%} chance of being spam")

---

## 🔴 Step 7: Perform the Evasion Attack

Now the real attack begins.

We use ART's **HopSkipJump** attack. This is a black-box evasion attack — the attacker only needs to send inputs and get predictions back. They never see inside the model. This is realistic because real attackers usually cannot see the model they are attacking.

**What it does:** Takes spam messages the model correctly identifies, makes tiny mathematical adjustments to their numerical vectors, and keeps adjusting until the model classifies them as legitimate.

In [ ]:
# =============================================================================
# WRAP THE MODEL IN ART AND SELECT SPAM SAMPLES TO ATTACK
# =============================================================================

art_classifier = SklearnClassifier(model=model, clip_values=(0, 1))

# Select spam messages the model currently gets right
# No point attacking messages the model already gets wrong
spam_indices = np.where((y_test == 1) & (model.predict(X_test) == 1))[0]
attack_sample = X_test[spam_indices[:5]]

print(f"Selected {len(attack_sample)} correctly-identified spam messages to attack.")
print("Model currently classifies ALL of these as SPAM.")
print("")
print("Launching evasion attack...")
print("(This may take 1-2 minutes)")

In [ ]:
# =============================================================================
# RUN THE HOPSKIPJUMP EVASION ATTACK
# =============================================================================
# targeted=False : just fool the model, not force a specific prediction
# max_iter=20    : up to 20 rounds of adjustments
# max_eval=100   : maximum model queries per iteration
# =============================================================================

attack = HopSkipJump(
    classifier=art_classifier,
    targeted=False,
    max_iter=20,
    max_eval=100,
    init_eval=10
)

x_adv = attack.generate(x=attack_sample)

print("Attack complete!")
print("")

original_predictions = model.predict(attack_sample)
adversarial_predictions = model.predict(x_adv)

print("Results:")
print("=" * 55)
for i in range(len(attack_sample)):
    orig = "SPAM" if original_predictions[i] == 1 else "HAM"
    adv  = "SPAM" if adversarial_predictions[i] == 1 else "HAM"
    fooled = "✓ FOOLED" if adversarial_predictions[i] == 0 else "✗ Still detected"
    print(f"Message {i+1}: Original={orig} | After attack={adv} | {fooled}")

print("")
success_rate = np.mean(adversarial_predictions == 0) * 100
print(f"Attack success rate: {success_rate:.0f}% of spam messages evaded detection")

### 👀 What Do You See?

- How many spam messages successfully evaded the filter?
- The attack modified numerical vectors. What does this tell you about how spam filters can be fooled?
- In a real-world scenario, what would an attacker do with this capability?

### 🧪 Try This — Change the Attack Strength

Go back to the attack cell and change `max_iter=20` to `max_iter=5`, then rerun.

- Does a weaker attack still successfully evade the filter?
- Then try `max_iter=50`. Does more always mean better?

---

## 💭 Step 8: Reflect

In [ ]:
# =============================================================================
# REFLECTION - edit your answers and run to save
# =============================================================================

reflection = """
LAB 1 - EVASION ATTACK REFLECTION
===================================

Q1: In plain English, what is an evasion attack?
A1: [TYPE YOUR ANSWER HERE]

Q2: In a real text-based attack, what kinds of changes might an attacker
    make to a spam email to evade a filter?
A2: [TYPE YOUR ANSWER HERE]

Q3: What is one defensive measure that could reduce evasion attack effectiveness?
A3: [TYPE YOUR ANSWER HERE]

Q4: Name one real-world system other than spam where an evasion attack
    could cause serious harm.
A4: [TYPE YOUR ANSWER HERE]
"""

with open('../outputs/Lab1_Reflection.txt', 'w') as f:
    f.write(reflection)

print("Reflection saved to outputs/Lab1_Reflection.txt")
print(reflection)

---

## ✅ Lab 1 Complete

You have successfully trained a spam filter and performed an evasion attack against it using ART.

Return to [START_HERE.ipynb](START_HERE.ipynb) and open Lab 2 — Poisoning Attack.

---
*Built with the Adversarial Robustness Toolbox (ART) — https://github.com/Trusted-AI/adversarial-robustness-toolbox*